In [ ]:
import os
import pandas as pd
import re

In [ ]:
target = "NepQuake15/RawNepQuake15/"
file_list = [os.path.join(target,x) for x in os.listdir(target)]
file_list.sort()
file_list

In [ ]:
#### CLEANING ##### remove @usernames and external-urls
def cleanText(pdf):
    pdf['Text'] = pdf.apply(lambda x: re.sub('@[\w\d]+','',x['Text']),axis=1) # remove @usernames
    # pdf['Text'] = pdf.apply(lambda x: re.sub('#[a-zA-Z]+','',x['Text']),axis=1) # remove hash-tag
    pdf['Text'] = pdf.apply(lambda x: re.sub('\w+:\/\/[\w.\/]+',' ',x['Text']),axis=1)  # remove external urls
    # pdf['Text'] = pdf.apply(lambda x: re.sub('[a-zA-Z]+','',x['Text']),axis=1)  # remove english alphabets
    pdf['Text'] = pdf.apply(lambda x: re.sub("^\s[-_]\s*|\s*-\s*$",'',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\.+",'.',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub(",+",',',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\?+",'?',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("!+",'!',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\*+",'*',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("।+",'।',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub('"+','"',x['Text']),axis=1)
    pdf['Text'] = pdf.apply(lambda x: re.sub("\n+","\n",re.sub('( +\n +)+','\n',x['Text'])),axis=1) # remove new line feeds and extra spaces
    pdf['Text'] = pdf.apply(lambda x: re.sub(" +"," ",x['Text']).strip(),axis=1) # removing spaces
    return pdf


In [ ]:
agg = pd.DataFrame()

In [ ]:
all_total = 0
for file in file_list:
    print(file)
    pdf = pd.read_csv(file)
    print('before',pdf.shape)
    all_total += pdf.shape[0]
    pdf = cleanText(pdf)
    
    pdf.index = range(pdf.shape[0])
    
    ### Checking for shortest string in the dataframe
    lengths = pdf.apply(lambda x: len(x['Text'].split(" ")),axis=1)
    print("Min",min(lengths),"Max", max(lengths),"Avg",sum(lengths)/lengths.shape[0])
    
    ##### CLEANING ##### remove short sentences ----
    indexes = pdf.apply(lambda x: len(x['Text'].split(" "))>=5,axis=1)
    pdf = pdf[indexes]
    
    ### Drop Duplicates
    pdf = pdf.drop_duplicates()
    print("after",pdf.shape)
    
    if not agg.empty:
        agg = pd.concat([agg,pdf])
    else:
        agg = pdf
    
agg = agg.drop_duplicates()
print("Grand Total: ",all_total)

In [ ]:
agg = agg.sort_values(by=['Date'])
agg['Date'].value_counts(), agg.shape

In [ ]:
agg.to_csv("NepQuake15/finalized.csv",index=False)

In [ ]:
test = agg[agg.apply(lambda x: len(x['Text'].split(" "))>=10,axis=1)]
test = test.sort_values(by=['Date'])
test['Date'].value_counts(), test.shape

In [ ]:
test.to_csv("NepQuake15/finalized.csv",index=False)